In [ ]:
import mysql.connector 
import pandas as pd
from pandas import DataFrame
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import config
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split, cross_val_score
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from IPython.display import Image
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database= 'Dog_Breeds'
)
cursor = cnx.cursor()

# cursor.close()
# cnx.close()

In [ ]:
sq= """SELECT * FROM Dogs"""
cursor.execute(sq)
sql=cursor.fetchall()

In [ ]:
#Naming Columns
df= pd.DataFrame(sql)
df.columns = ["Name", "Male_Min_Weight", "Female_Min_Weight", "Male_Max_Weight", "Female_Max_Weight","Male_Height", "Female_Height", "Exercise_Min_Min_Daily", 
              "Exercise_Max_Min_Daily",
    "Energy_Level", "Longevity_Min", "Longevity_Max",
              "Tend_Drool", "Tend_Snore", "Tend_Bark", "Tend_Dig", "Attention_Needs", "Grooming_Needs", "AKC_Class", "UKC_Class", "Prevalence"]

In [ ]:
# run code Twice, once with non AKC as a labaled "Not AKC classified."
df=df[df.AKC_Class != '<p> <strong>UKC Classification</strong>']

# Data Exploration

In [ ]:
sns.set_style('darkgrid')

for column in list(df.columns):
    plt.figure(figsize = (10,5))
    sns.countplot(df[column], alpha =.80)
#     plt.title('Survivors vs Non-Survivors')
    plt.ylabel('# counts')
    plt.show()

# Cleaning columns

In [ ]:
# Changing low medium high ranking to 0 ,1 ,2 respectivly

df['Tend_Drool'] = df['Tend_Drool'].map({'Low': 0, 'Moderate': 1, "High": 2})
df['Tend_Snore'] = df['Tend_Snore'].map({'Low': 0, 'Moderate': 1, "High": 2})
df['Tend_Dig'] = df['Tend_Dig'].map({'Low': 0, 'Moderate': 1, "High": 2})
df['Attention_Needs'] = df['Attention_Needs'].map({'Low': 0, 'Moderate': 1, "High": 2})
df['Tend_Bark'] = df['Tend_Bark'].map({'Low': 0, 'Moderate': 1, "High": 2, "Unknown": 1})
df['Grooming_Needs'] = df['Grooming_Needs'].map({'Low': 0, 'Moderate': 1, "High": 2, "Unknown": 1})

# breeds with years were checked by hand. terrier and working respectivly
df['Energy_Level'] = df['Energy_Level'].map({'Very energet': 2, 'Average': 1, "Laid back ": 0, "Very Energet": 2, "Average.": 1, "high": 2, 
                                             "Laid Back": 0, "Moderate": 1, "Active": 2, "12-14 yrs": 1, "Bred to work": 2, "High": 2, "low to moder": 0, "10-13 yrs.": 2 })                                                                         
                                             
df['Prevalence'] = df['Prevalence'].map({'Common': 0, 'So-so': 1, "Rare": 2, "Common</p>": 0, "So-so</p>.": 1, "Common.": 0,
                                         
     "So-So.": 1, 
    "So-so.": 1, "Rare.": 2, "So-so.</p>": 1, "Rare</p>": 2})    

#Mergeing Identical columns, final cleaning

df=df.replace(to_replace ="Non-Sporting", 
                 value ="Non-sporting") 
df=df.replace(to_replace ="companion", 
                 value ="Non-sporting") 
df=df.replace(to_replace ="Working.", 
                 value ="Working") 

df=df.replace(to_replace ="Toy.", 
                 value ="Toy") 


df=df.replace(to_replace ="Terrier.", 
                 value ="Terrier") 


df=df.replace(to_replace ="Hound.", 
                 value ="Hound")    

df= df.drop(['UKC_Class', "Name"], axis=1)

In [ ]:
#Grouping AKC breeds inorder to reduce classes, scores proved unable to predict difference between group 3 inputs, so merged them into 1.
group1=Toy
group2=Working, Sporting
group3=Terrier, Herding, Hound, Non-sporting

In [ ]:
df['target']=['group1' if i =="Toy" else "group2" if i == "Working" or i == "Sporting" else "group3" for i in df.AKC_Class]

In [ ]:
df= df.drop("AKC_Class", axis=1)

In [ ]:
df=df.fillna(1)

In [ ]:
#Setting X and Y data for all models.
Y= df.target
X= df.drop("target", axis=1)
for i in list(X.columns):
    X[i].astype(float)

In [ ]:
#Splitting Data into test train, 20% test size.
X_train, X_test, y_train, y_test= train_test_split(X,Y, test_size=0.2, random_state=2)

In [ ]:
#Functions for evaluating

def evaluate(test, pred, model):
    return [model, precision_score(test, pred, average= None), recall_score(test, pred, average= None), accuracy_score(test, pred), f1_score(test, pred, average= None)]
# print metrics
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average= None)))
    print("Recall Score: {}".format(recall_score(labels, preds, average= None)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average= None)))
# plot confusion metrix
def conf_metrix(model):
    # Create the basic matrix
    plt.imshow(model,  cmap=plt.cm.Blues)
    # Add title and axis labels
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # Add appropriate axis scales
    class_names = set(y) # Get class labels to add to matrix
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    # Add labels to each cell
    thresh = model.max() / 2. # Used for text coloring below
    # Here we iterate through the confusion matrix and append labels to our visualization
    for i, j in itertools.product(range(model.shape[0]), range(model.shape[1])):
            plt.text(j, i, model[i, j],
                     horizontalalignment='center',
                     color='white' if model[i, j] > thresh else 'black')
    # Add a legend
    plt.colorbar()
    plt.show()

# Decision Tree

In [ ]:
tree_clf= DecisionTreeClassifier(max_depth= 10)
tree_clf.fit(X_train, y_train)

In [ ]:
tree_clf_pred= clf.predict(X_test)

In [ ]:
def gridsearchCV(model):
    param_grid={
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 10, 12],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 3, 4, 5, 6]}
    return GridSearchCV(model,param_grid, cv=3)

In [ ]:
tree_clf_grid=gridsearchCV(tree_clf)
tree_clf_grid.fit(X_train, y_train)

In [ ]:
tree_clf_grid.best_params_

In [ ]:
tree_clf_grid_pred= tree_clf_grid.predict(X_test)

In [ ]:
evaluate(y_test, tree_clf_grid_pred)

In [ ]:
# Create DOT data
dot_data = export_graphviz(clf, out_file=None,
                           feature_names=X_train.columns,
                           class_names=np.unique(Y).astype('str'),
                           filled=True, rounded=True, special_characters=True)
# Draw graph
graph = graph_from_dot_data(dot_data)
# Show graph
Image(graph.create_png())

# KNN

In [ ]:
# Instantiate StandardScaler
scaler = StandardScaler().fit(X_train)

# Transform the training and test sets
scaled_data_train = scaler.transform(X_train)
scaled_data_test = scaler.transform(X_test)

# Convert into a DataFrame
scaled_df_train = pd.DataFrame(scaled_data_train)

In [ ]:
# Instantiate KNeighborsClassifier
Kclf = KNeighborsClassifier(n_neighbors= 7)

# Fit the classifier
Kclf.fit(scaled_data_train, y_train)

# Predict on the test set
test_preds = Kclf.predict(scaled_data_test)

KNN_eva=evaluate(y_test, test_preds, 'KNN')

In [ ]:
#Function to find the best KNN formula w/ Micro averaging
def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1, 1):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        f1 = f1_score(y_test, preds, average="micro")
        if f1 > best_score:
            best_k = k
            best_score = f1
    print("Best Value for k: {}".format(best_k))
    print("F1-Score: {}".format(best_score))

In [ ]:
find_best_k(scaled_data_train, y_train, scaled_data_test, y_test)

# Bagging Trees Forrest

In [ ]:
sds=clf.feature_importances_
def plot_feature_importances(model):
    n_features = X_train.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

plot_feature_importances(clf)

In [ ]:
pred = clf.predict(X_test)

# Confusion matrix and classification report
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
print("Testing Accuracy for Decision Tree Classifier: {:.4}%".format(accuracy_score(y_test, pred) * 100))

In [ ]:
# Instantiating an BaggingClassifier
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=7), 
                                 n_estimators=20)

In [ ]:
bagged_tree.fit(X_train, y_train)

In [ ]:
bagg_pred= bagged_tree.predict(X_test)
bagg_pred

In [ ]:
Baggin_eva=evaluate(y_test, bagg_pred, 'Baggin')
Baggin_eva

# Random Forrest

In [ ]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth= 7)
forest.fit(X_train, y_train)

In [ ]:
forest_pred=forest.predict(X_test)

In [ ]:
Forrests_eva=evaluate(y_test, forest_pred, 'Random Forrests')

# Adaboost and Gradient Boost


In [ ]:
# Instantiate an AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(random_state=42)

# Instantiate an GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(random_state=42)

In [ ]:
adaboost_clf.fit(X_train, y_train)
gbt_clf.fit(X_train, y_train)

In [ ]:
# AdaBoost model predictions
adaboost_train_preds = adaboost_clf.predict(X_train)
adaboost_test_preds = adaboost_clf.predict(X_test)

# GradientBoosting model predictions
gbt_clf_train_preds = gbt_clf.predict(X_train)
gbt_clf_test_preds = gbt_clf.predict(X_test)

In [ ]:
ADA_eva=evaluate(y_test, adaboost_test_preds, 'AdaBoost')
GBT_eva=evaluate(y_test, gbt_clf_test_preds, 'GBT')

In [ ]:
#Function to test Adaboost and gbt boost test vs train scores
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average='micro')
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    
print("Training Metrics")
display_acc_and_f1_score(y_train, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_train, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("Testing Metrics")
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

In [ ]:
adaboost_confusion_matrix = confusion_matrix(y_test, adaboost_test_preds)
adaboost_confusion_matrix

In [ ]:
gbt_confusion_matrix = confusion_matrix(y_test, gbt_clf_test_preds)
gbt_confusion_matrix

In [ ]:
adaboost_classification_report = classification_report(y_test, adaboost_test_preds)
print(adaboost_classification_report)

In [ ]:
gbt_classification_report = classification_report(y_test, gbt_clf_test_preds)
print(gbt_classification_report)

In [ ]:
print('Mean GBT Cross-Val Score (k=5):')
print(cross_val_score(gbt_clf, X, Y, cv=5).mean())

# XGBOOST

In [ ]:
# Instantiate XGBClassifier
XG_clf = XGBClassifier()

# Fit XGBClassifier
XG_clf.fit(X_train, y_train)

# Predict on training and test sets
training_preds = XG_clf.predict(X_train)
XG_test_preds = XG_clf.predict(X_test)

# Accuracy of training and test sets
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [7],
    'subsample': [0.5, 0.7],
    'n_estimators': [100],
}

In [ ]:
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
test_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('')
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
XGBoost_eva=evaluate(y_test, XG_test_preds, 'XGBoost')
XGBoost_eva

# SVM

In [ ]:
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

In [ ]:
svm_pred= svm_clf.predict(X_test)

In [ ]:
print_metrics(y_test, svm_pred)
SVM_eva=evaluate(y_test, svm_pred, 'SVM')

# Logistical Regression

In [ ]:
min_max = MinMaxScaler().fit(X_train)
X_train_sca = min_max.transform(X_train)
X_test_sca = min_max.transform(X_test)

# created model
# check the running time
import time
start = time.time()
log_clf = LogisticRegression()
log_clf.fit(X_train_sca, y_train)
log_runtime = time.time() - start

# prediction of  the test set
log_pred = log_clf.predict(X_test_sca)

# evaluated metrix for result table
log_eva = evaluate(y_test, log_pred, 'Logistic Regression ')

# Results

In [ ]:
#Building a matrix of all test data
log_eva = evaluate(y_test, log_pred, 'Logistic Regression ')
Tree_eva=evaluate(y_test, clf_grid_pred, 'Decision Tree')
Baggin_eva=evaluate(y_test, bagg_pred, 'Baggin')
Forrests_eva=evaluate(y_test, forest_pred, 'Random Forrests')
KNN_eva=evaluate(y_test, test_preds, 'KNN')
ADA_eva=evaluate(y_test, adaboost_test_preds, 'AdaBoost')
GBT_eva=evaluate(y_test, gbt_clf_test_preds, 'GBT')
XGBoost_eva=evaluate(y_test, XG_test_preds, 'XGBoost')
SVM_eva=evaluate(y_test, svm_pred, 'SVM')

In [ ]:
results= pd.DataFrame()

In [ ]:
results= results.append([log_eva, Tree_eva, Baggin_eva, Forrests_eva, KNN_eva, ADA_eva, 
                         GBT_eva, XGBoost_eva, SVM_eva])

In [ ]:
results.columns=["Model", "Precision Score", "Recall Score", "Accuracy Score", "F1 Score"]
results

In [ ]:
#getting matrix with Weighted Scores
def evaluate_weighted(test, pred, model):
    return [model, precision_score(test, pred, average= "weighted"), recall_score(test, pred, average= "weighted"),
            accuracy_score(test, pred), f1_score(test, pred, average= "weighted")]

In [ ]:
log_eva2 = evaluate_weighted(y_test, log_pred, 'Logistic Regression ')
Tree_eva2=evaluate_weighted(y_test, clf_grid_pred, 'Decision Tree')
Baggin_eva2=evaluate_weighted(y_test, bagg_pred, 'Baggin')
Forrests_eva2=evaluate_weighted(y_test, forest_pred, 'Random Forrests')
KNN_eva2=evaluate_weighted(y_tesresults_weighted= pd.DataFrame()t, test_preds, 'KNN')
ADA_eva2=evaluate_weighted(y_test, adaboost_test_preds, 'AdaBoost')
GBT_eva2=evaluate_weighted(y_test, gbt_clf_test_preds, 'GBT')
XGBoost_eva2=evaluate_weighted(y_test, XG_test_preds, 'XGBoost')
SVM_eva2=evaluate_weighted(y_test, svm_pred, 'SVM')

In [ ]:
results_weighted= pd.DataFrame()

In [ ]:
results_weighted= results_weighted.append([log_eva2, Tree_eva2,
                                           Baggin_eva2, Forrests_eva2, KNN_eva2, ADA_eva2, GBT_eva2, 
                                           XGBoost_eva2, SVM_eva2])


In [ ]:
results_weighted.columns=["Model", "Precision Score", "Recall Score", "Accuracy Score", "F1 Score"]

In [ ]:
results_weighted